https://factsbeyondnumbers.wordpress.com/2020/06/18/extracting-daily-stock-price-data-in-python/

download the scanner csv from chartink( only premium chartink gives all historical data, free users can get only 9 months  limited data)
download all historical data using this python script to know next day or previous day price action
check the outcome using this script and get the result of backtest

In [23]:
# Import modules
from nsepy import get_history
from datetime import datetime, timedelta
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like #For solving import pandas_datareader issue
import numpy as np
import datetime
import csv
import requests
import pandas_datareader.data as web
import pandas_datareader as pdr
from pandas_datareader import data, wb
from datetime import date
#to plot within notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
path = r"A:\FinMinII\Trading\NSE_Indices\Backtest\Nifty_Backtest"
file = "Backtest RSI and Nifty.csv"#


# # # Input Start and End Date
start = datetime.datetime(2011,1,7) # keep the strt date starting from Friday for weekly analysis (Friday is first day for expirty week and month)
end = pd.to_datetime('today').normalize() #datetime.datetime(2022,2,2)

print(start)
print(end)

2011-01-07 00:00:00
2022-03-08 00:00:00


In [3]:
import os
os.getcwd()
os.chdir(path)
os.getcwd()

'A:\\FinMinII\\Trading\\NSE_Indices\\Backtest\\Nifty_Backtest'

In [4]:
NIFTY_Org = get_history(symbol="NIFTY",
                        start=start,
                        end=end,
                        index=True)



In [5]:
NIFTY_DF = NIFTY_Org.reset_index(level=['Date'])
NIFTY_DF['Date'] = pd.to_datetime(NIFTY_DF['Date'])


# Nifty Daily Analysis from NIFTY_Org Data

In [6]:
# drop rows from Nifty Dataframe where close price is not available, possibelly they are holidays.
NIFTY_Dly = NIFTY_DF.copy()
NIFTY_Dly.dropna(subset = ["Close"], inplace=True)

In [7]:
# Calculating moving average, below is for 124 period to calculate SRT rest also can be calculated
NIFTY_Dly['124_DSMA'] = NIFTY_Dly['Close'].rolling(window=124).mean()


NIFTY_Dly.tail(2)

,Date,Open,High,Low,Close,Volume,Turnover,124_DSMA
2765,2022-03-07,15867.95,15944.60,15711.45,15863.15,NaN,NaN,17529.058871
2766,2022-03-08,15747.75,16028.75,15671.45,16013.45,NaN,NaN,17518.182661


In [8]:
# Calculating SRT for each day
NIFTY_Dly['SRT'] = NIFTY_Dly['Close']/NIFTY_Dly['124_DSMA']

NIFTY_Dly.tail(2)


,Date,Open,High,Low,Close,Volume,Turnover,124_DSMA,SRT
2765,2022-03-07,15867.95,15944.60,15711.45,15863.15,NaN,NaN,17529.058871,0.904963
2766,2022-03-08,15747.75,16028.75,15671.45,16013.45,NaN,NaN,17518.182661,0.914105


# Nifty Weekly Analysis from NIFTY_Org Data

In [9]:
# # Creating Historical date data frame to handle holidays

# DS = pd.date_range(start=start, end=end)
# Nifty_Historical_DF = pd.DataFrame({'Date': DS})

In [10]:
NIFTY_Wkly = NIFTY_DF.copy()

In [11]:
NIFTY_Wkly.head(2)

,Date,Open,High,Low,Close,Volume,Turnover
0,2011-01-07,6030.9,6051.20,5883.60,5904.60,171809106.0,8.325790e+10
1,2011-01-10,5901.3,5907.25,5740.95,5762.85,156945855.0,7.920380e+10


In [12]:
# Getting week number
NIFTY_Wkly['Week_Number'] = NIFTY_Wkly['Date'].dt.week
# Getting year. Weeknum is common across years to we need to create unique index by using year and weeknum
NIFTY_Wkly['Year'] = NIFTY_Wkly['Date'].dt.year

NIFTY_Wkly['Day_of_Week'] = NIFTY_Wkly['Date'].dt.day_name()

# Grouping based on required values
NIFTY_Wkly_Grp = NIFTY_Wkly.groupby(['Year','Week_Number']).agg({'Year':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
NIFTY_Wkly_Grp.tail(2)

C:\Users\Anurag\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


Year      Open      High       Low     Close        Volume
Year Week_Number                                                            
2022 9            2022  16481.60  16815.90  16133.80  16245.35  1.378081e+09
     10           2022  15867.95  16028.75  15671.45  16013.45  0.000000e+00

In [13]:
# Calculating moving average, below is for 124 period to calculate SRT rest also can be calculated
NIFTY_Wkly_Grp['124_DSMA'] = NIFTY_Wkly_Grp['Close'].rolling(window=124).mean()

# Calculating SRT for each day
NIFTY_Wkly_Grp['SRT'] = NIFTY_Wkly_Grp['Close']/NIFTY_Wkly_Grp['124_DSMA']

NIFTY_Wkly_Grp.tail(2)

Year      Open      High       Low     Close        Volume  \
Year Week_Number                                                               
2022 9            2022  16481.60  16815.90  16133.80  16245.35  1.378081e+09   
     10           2022  15867.95  16028.75  15671.45  16013.45  0.000000e+00   

                      124_DSMA       SRT  
Year Week_Number                          
2022 9            13745.516935  1.181865  
     10           13778.765726  1.162183

In [ ]:
# NIFTY_opt.to_csv("OP_NIFTY_Historical_data.csv")
# Nifty_Weekly_Close.to_csv("Nifty_Weekly_Close.csv")

In [15]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Nifty_EDA.xlsx', engine='xlsxwriter')

In [16]:
# Write each dataframe to a different worksheet.
NIFTY_Dly.to_excel(writer, sheet_name='NIFTY_Dly')
NIFTY_Wkly_Grp.to_excel(writer, sheet_name='NIFTY_Wkly')
# df3.to_excel(writer, sheet_name='Sheet3')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [22]:
from nsepy.history import get_price_list
prices = get_price_list(dt=date(2015,1,1))
print(prices)

          SYMBOL SERIES     OPEN     HIGH      LOW    CLOSE     LAST  \
0      20MICRONS     EQ    30.45    31.00    30.25    30.40    30.40   
1     3IINFOTECH     EQ     9.50    10.10     9.25     9.75     9.80   
2        3MINDIA     EQ  6444.80  6529.15  6402.90  6504.60  6462.00   
4        8KMILES     EQ   633.90   739.95   630.00   727.30   725.00   
5       A2ZINFRA     EQ    17.10    18.00    17.05    18.00    18.00   
...          ...    ...      ...      ...      ...      ...      ...   
1571   ZODJRDMKJ     EQ    46.00    48.95    45.20    45.55    45.20   
1572       ZUARI     EQ   260.50   270.00   258.00   267.30   266.85   
1573   ZUARIGLOB     EQ   102.80   104.20   102.50   102.95   103.10   
1574   ZYDUSWELL     EQ   819.00   826.85   808.85   811.40   810.00   
1575       ZYLOG     EQ     6.60     6.80     6.50     6.70     6.80   

      PREVCLOSE  TOTTRDQTY     TOTTRDVAL    TIMESTAMP  TOTALTRADES  \
0         30.60      37604  1.150221e+06  01-JAN-2015          20